In [298]:
import csv
from math import log
import gensim.parsing.preprocessing as prep
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
import numpy as np
import itertools
from math import log
from sklearn.model_selection import StratifiedKFold

In [185]:
def str2subset(s, l):
    return [s[i:i+l] for i in range(len(s)-l+1)]


def str2subsets_dict(s, min_l, max_l):
    assert max_l <= len(s)
    sets = {}
    for i in range(min_l, max_l+1):
        sets[i] = str2subset(s, i)
#     return list(itertools.chain.from_iterable(list(sets.values())))
    return sets


def str2subsets_str(s, min_l, max_l):
    assert max_l <= len(s)
    sets = []
    for i in range(min_l, max_l+1):
        sets += str2subset(s, i)
    return ' '.join(sets)


def get_docs(f, upper, min_len = 50):
    s = ''
    c = f.read(1)
    while c:
        if c not in 'acgtACGT':
            c = f.read(1)
            continue
        if c.isupper() != upper:
            break
        s += c
        c = f.read(1)
    docs = []
    k = 0
    while k+min_len < len(s):
        docs.append(s[k:k+min_len])
        k += min_len
    if len(docs) > 0:
        docs[-1] += s[k:-1]
    else:
        docs.append(s)
    return docs, c


def make_docs(filePath):
    exons = []
    introns = []
    f = open(filePath)
    c = f.read(1)
    while (c):
        upper = c.isupper()
        pair = get_docs(f, upper, 200)
        doc = c + pair[0][0]
        if upper:
            exons.append(doc)
        else:
            introns.append(doc)
        for i in range(1, len(pair[0])):
            doc = pair[0][i]
            if upper:
                exons.append(doc)
            else:
                introns.append(doc)
        c = pair[1]
    return exons, introns


def get_subset_docs(filePath, min_l, max_l):
    exons, introns = make_docs(filePath)
    exons = [str2subsets_str(s, min_l, max_l) for s in exons]
    introns = [str2subsets_str(s, min_l, max_l) for s in introns]
    return exons, introns

In [186]:
exons, introns = get_subset_docs(r"C:\Users\admin\PycharmProjects\dna\files\dna.txt", 1, 7)

In [166]:
data = pd.DataFrame(data=np.array([exons+introns,
                          np.concatenate((np.ones_like(exons, dtype=np.int8),
                                          np.zeros_like(introns, dtype=np.int8))).tolist()]).T,
                    columns=['dna', 'exon'])

In [292]:
data.head()

,dna,exon
0,G T G G C G C G A G C T T C T G A A A C T A G ...,1
1,A C T T A T T T A C C A A G C A T T G G A G G ...,1
2,A T T T A G G A C C A A T A A G T C T T A A T ...,1
3,T A C G A A C C A A A C C T A T T T A A A A C ...,1
4,G A A G G A A T G T T C C C A A T A G T A G A ...,1


In [168]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
])

In [169]:
kf = KFold(n_splits=10, shuffle=True)
f1_scores = []
for train, test in kf.split(data):
    X = data['dna'][train]
    Y = data['exon'][train]
    text_clf.fit(data['dna'], data['exon'])
    predicted = text_clf.predict(data['dna'][test])
    f1_scores.append(f1_score(data['exon'][test], predicted, average='micro'))
print("F-score: ", sum(f1_scores)/len(f1_scores))

C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
C:\Program Files

F-score:  0.876006464883926


C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [187]:
important_features = []
with open(r"C:\Users\admin\PycharmProjects\dna\files\important.features.deep-7.m-100.prob-find-0.999.prob-90.0.threshold-2.0.txt", 'r') as f:
    for line in f:
        if line[0] != 'e':
            important_features.append(line.split()[0])

In [188]:
exons_only_important = []
for s in exons:
    s_imp = []
    for ss in s.split():
        if ss.lower() in important_features:
            s_imp.append(ss)
    exons_only_important.append(' '.join(s_imp))
    
introns_only_important = []
for s in introns:
    s_imp = []
    for ss in s.split():
        if ss in important_features:
            s_imp.append(ss)
    introns_only_important.append(' '.join(s_imp))

In [190]:
data_imp = pd.DataFrame(data=np.array([exons_only_important+introns_only_important,
                          np.concatenate((np.ones_like(exons_only_important, dtype=np.int8),
                                          np.zeros_like(introns_only_important, dtype=np.int8))).tolist()]).T,
                    columns=['dna', 'exon'])
data_imp.head()

,dna,exon
0,TGGG TAGGC CACTG GAAGC TTTTT AAAGA AAGAA TTCTG...,1
1,AAAGA TTTTT TTTTT TTTTT ATCCAA TTTTTT TTTTAA C...,1
2,GAAGA AAGAA GAAGC GAAGA AAGAA TACGA ACGAA AAAG...,1
3,TCCC GAAAA CAGATGA,1
4,AAAGA GTTCTAC ATAAGTC,1


In [191]:
f1_scores_imp = []
for train, test in kf.split(data_imp):
    X = data_imp['dna'][train]
    Y = data_imp['exon'][train]
    text_clf.fit(data_imp['dna'], data_imp['exon'])
    predicted = text_clf.predict(data_imp['dna'][test])
    f1_scores_imp.append(f1_score(data_imp['exon'][test], predicted, average='micro'))
print("F-score: ", sum(f1_scores_imp)/len(f1_scores_imp))

C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
C:\Program Files

F-score:  0.9686411149825783


C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [295]:
def get_f_score(prob, treshold):
    important_features = []
    with open(r"C:\Users\admin\PycharmProjects\dna\files\important.features.deep-7.m-100.prob-find-0.999."
              +"prob-"+prob
              +".threshold-"+treshold
              +".txt", 'r') as f:
        for line in f:
            if line[0] != 'e':
                important_features.append(line.split()[0])
                
    exons_only_important = []
    for s in exons:
        s_imp = []
        for ss in s.split():
            if ss.lower() in important_features:
                s_imp.append(ss)
        exons_only_important.append(' '.join(s_imp))

    introns_only_important = []
    for s in introns:
        s_imp = []
        for ss in s.split():
            if ss in important_features:
                s_imp.append(ss)
        introns_only_important.append(' '.join(s_imp))
        
    data_imp = pd.DataFrame(data=np.array([exons_only_important+introns_only_important,
                          np.concatenate((np.ones_like(exons_only_important, dtype=np.int8),
                                          np.zeros_like(introns_only_important, dtype=np.int8))).tolist()]).T,
                    columns=['dna', 'exon'])
    
#     _clf = Pipeline([('vect', CountVectorizer()),
#                      ('tfidf', TfidfTransformer()),
#                      ('clf', MultinomialNB())])
    f1_scores_imp = []
    for train, test in kf.split(data_imp):
        X = data_imp['dna'][train]
        Y = data_imp['exon'][train]
        text_clf.fit(data_imp['dna'], data_imp['exon'])
        predicted = text_clf.predict(data_imp['dna'][test])
        f1_scores_imp.append(f1_score(data_imp['exon'][test], predicted, average='micro'))
        
    return sum(f1_scores_imp)/len(f1_scores_imp), text_clf
#     return np.mean(cross_val_score(text_clf, data_imp.dna, data_imp.exon, cv=5, n_jobs=4)), text_clf

In [ ]:
f_scores = []
prob_treshold_dict = {}
with open(r"C:\Users\admin\PycharmProjects\dna\files\output_probs_nh.txt", 'r') as prob_treshold_f:
    for line in prob_treshold_f:
        prob = line.split()[0]
        treshold = line.split()[1]
        prob_treshold_dict.setdefault(float(prob), []).append(float(treshold))
        f_scores.append([float(prob), float(treshold), get_f_score(prob, treshold)[0]])

In [200]:
f_scores

[[70.0, 1.5, 0.8550522648083623],
 [70.0, 1.75, 0.8597560975609756],
 [70.0, 2.0, 0.862253193960511],
 [70.0, 2.25, 0.9710220673635307],
 [70.0, 2.5, 0.9830429732868756],
 [70.0, 2.75, 0.9806620209059232],
 [70.0, 3.0, 0.9879790940766551],
 [71.0, 1.5, 0.8546457607433217],
 [71.0, 1.75, 0.8599303135888501],
 [71.0, 2.0, 0.9541811846689896],
 [71.0, 2.25, 0.975725900116144],
 [71.0, 2.5, 0.9807200929152149],
 [71.0, 2.75, 0.9807200929152149],
 [71.0, 3.0, 0.9903600464576074],
 [72.0, 1.5, 0.8552845528455284],
 [72.0, 1.75, 0.8573170731707316],
 [72.0, 2.0, 0.963937282229965],
 [72.0, 2.25, 0.975725900116144],
 [72.0, 2.5, 0.9830429732868758],
 [72.0, 2.75, 0.9854819976771196],
 [72.0, 3.0, 0.9904181184668989],
 [73.0, 1.5, 0.8552845528455284],
 [73.0, 1.75, 0.9349593495934959],
 [73.0, 2.0, 0.9685830429732867],
 [73.0, 2.25, 0.9783391405342625],
 [73.0, 2.5, 0.9829849012775842],
 [73.0, 2.75, 0.9807200929152149],
 [74.0, 1.5, 0.8842624854819977],
 [74.0, 1.75, 0.9419279907084785],
 [74.

In [285]:
prob_treshold_dict = {}
with open(r"C:\Users\admin\PycharmProjects\dna\files\output_probs_nh.txt", 'r') as prob_treshold_f:
    for line in prob_treshold_f:
        prob = line.split()[0]
        treshold = line.split()[1]
        prob_treshold_dict.setdefault(float(prob), []).append(float(treshold))

In [286]:
prob_treshold_dict

{50.0: [1.5,
  1.75,
  2.0,
  2.25,
  2.5,
  2.75,
  3.0,
  3.25,
  3.5,
  3.75,
  4.0,
  4.25,
  4.5,
  4.75,
  5.0,
  5.25,
  5.5,
  5.75,
  6.0,
  6.25],
 51.0: [1.5,
  1.75,
  2.0,
  2.25,
  2.5,
  2.75,
  3.0,
  3.25,
  3.5,
  3.75,
  4.0,
  4.25,
  4.5,
  4.75,
  5.0,
  5.25,
  5.5,
  5.75,
  6.0,
  6.25],
 52.0: [1.5,
  1.75,
  2.0,
  2.25,
  2.5,
  2.75,
  3.0,
  3.25,
  3.5,
  3.75,
  4.0,
  4.25,
  4.5,
  4.75,
  5.0,
  5.25,
  5.5,
  5.75,
  6.0],
 53.0: [1.5,
  1.75,
  2.0,
  2.25,
  2.5,
  2.75,
  3.0,
  3.25,
  3.5,
  3.75,
  4.0,
  4.25,
  4.5,
  4.75,
  5.0,
  5.25,
  5.5],
 54.0: [1.5,
  1.75,
  2.0,
  2.25,
  2.5,
  2.75,
  3.0,
  3.25,
  3.5,
  3.75,
  4.0,
  4.25,
  4.5,
  4.75,
  5.0,
  5.25],
 55.0: [1.5,
  1.75,
  2.0,
  2.25,
  2.5,
  2.75,
  3.0,
  3.25,
  3.5,
  3.75,
  4.0,
  4.25,
  4.5,
  4.75,
  5.0],
 56.0: [1.5,
  1.75,
  2.0,
  2.25,
  2.5,
  2.75,
  3.0,
  3.25,
  3.5,
  3.75,
  4.0,
  4.25,
  4.5,
  4.75],
 57.0: [1.5, 1.75, 2.0, 2.25, 2.5, 2.75, 3.0,

In [207]:
f_scores = []
for key, value in prob_treshold_dict.items():
    prob = str(key)
    treshold = str(max(value))
    f_scores.append([float(prob), float(treshold), get_f_score(prob, treshold)[0]])
f_scores

C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
C:\Program Files

[[50.0, 6.25, 0.8575493612078977],
 [51.0, 6.25, 0.8574912891986063],
 [52.0, 6.0, 0.8573751451800232],
 [53.0, 5.5, 0.8573751451800232],
 [54.0, 5.25, 0.8576074332171894],
 [55.0, 5.0, 0.8574332171893146],
 [56.0, 4.75, 0.8573751451800232],
 [57.0, 4.5, 0.8572009291521485],
 [58.0, 4.25, 0.8579558652729384],
 [59.0, 4.0, 0.8579558652729384],
 [60.0, 3.75, 0.8574912891986063],
 [61.0, 3.75, 0.8576074332171894],
 [62.0, 3.75, 0.9879790940766551],
 [63.0, 3.75, 0.9878629500580722],
 [64.0, 3.5, 0.9855400696864113],
 [65.0, 3.5, 0.9879790940766551],
 [66.0, 3.5, 0.9879210220673634],
 [67.0, 3.25, 0.9854819976771196],
 [68.0, 3.25, 0.9879790940766551],
 [69.0, 3.25, 0.990301974448316],
 [70.0, 3.0, 0.9880371660859465],
 [71.0, 3.0, 0.990301974448316],
 [72.0, 3.0, 0.9904181184668989],
 [73.0, 2.75, 0.9806039488966318],
 [74.0, 2.75, 0.987862950058072],
 [75.0, 2.75, 0.9854819976771196],
 [76.0, 2.75, 0.9854819976771196],
 [77.0, 2.5, 0.9854819976771196],
 [78.0, 2.5, 0.9831010452961673],
 

In [249]:
def get_entropy(p):
    assert p >=0 and p <= 1
    if p < 10e-10 or p > 1-10e-10:
        return 0
    return -p*log(p)-(1-p)*log(1-p)

def get_sum_entropy(clf, dnas):
    sum_len = sum([len(s) for s in dnas])
    probs = clf.predict_proba(dnas).T[0].tolist()
    h = 0
    for i in range(len(dnas)):
        h += get_entropy(probs[i])*float(len(dnas[i]))/sum_len
    return h

In [ ]:
from sklearn.base import clone
from copy import deepcopy
best_clf = deepcopy(get_f_score("72.0", "3.0")[1])

In [297]:
best_clf.predict_proba(data_imp.dna[5:10]).T[0].tolist()

C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


[0.9088677950253142,
 0.4839120712226538,
 0.8088248697175903,
 0.4474265224790244,
 0.7393356503051852]

In [299]:
exons, introns = make_docs(r"C:\Users\admin\PycharmProjects\dna\files\dna.txt")
data2 = pd.DataFrame(data=np.array([exons+introns,
                          np.concatenate((np.ones_like(exons, dtype=np.int8),
                                          np.zeros_like(introns, dtype=np.int8))).tolist()]).T,
                    columns=['dna', 'exon'])
data2.head()

,dna,exon
0,GTGGCGCGAGCTTCTGAAACTAGGCGGCAGAGGCGGAGCCGCTGTG...,1
1,ACTTATTTACCAAGCATTGGAGGAATATCGTAGGTAAAAATGCCTA...,1
2,ATTTAGGACCAATAAGTCTTAATTGGTTTGAAGAACTTTCTTCAGA...,1
3,GAAGGAATGTTCCCAATAGTAGACATAAAAGTCTTCGCACAGTGAA...,1
4,TCCTGTTGTTCTACAATGTACACATGTAACACCACAAAGAGATAAG...,1


In [304]:
skf = StratifiedKFold(n_splits=10, shuffle=True)
X = data2.dna
y = data2.exon